In [206]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql 
pymysql.install_as_MySQLdb()

In [207]:
Players = "../Desktop/nba-players-stats/Players.csv"
Players_df = pd.read_csv(Players)
Players_df.head()

,id,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [208]:
Season_Stats = "../Desktop/nba-players-stats/Seasons_Stats.csv"
Season_Stats_df = pd.read_csv(Season_Stats)
Season_Stats_df.head()

,id,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.71,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.71,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.70,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.56,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.55,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [209]:
new_players_df= Players_df[['Player', 'id','collage']].copy()
new_players_df.head()

,Player,id,collage
0,Curly Armstrong,0,Indiana University
1,Cliff Barker,1,University of Kentucky
2,Leo Barnhorst,2,University of Notre Dame
3,Ed Bartels,3,North Carolina State University
4,Ralph Beard,4,University of Kentucky


In [210]:
# Create a filtered dataframe from specific columns
new_players = ["Player","id", "collage"]
new_players_transformed= new_players_df[new_players].copy()

# Rename the column headers
new_players_transformed = new_players_transformed.rename(columns={"Player": "Player_Name",
                                                                  "id": "id",
                                                                  "collage": "college"})

# Clean the data by dropping duplicates and setting the index
new_players_transformed.drop_duplicates(inplace=True)
new_players_transformed.set_index('Player_Name', inplace=True)

new_players_transformed.head()

,id,college
Player_Name,,
Curly Armstrong,0,Indiana University
Cliff Barker,1,University of Kentucky
Leo Barnhorst,2,University of Notre Dame
Ed Bartels,3,North Carolina State University
Ralph Beard,4,University of Kentucky


In [211]:
New_Season_Stats_df = Season_Stats_df[['Player','id', 'G', 'FG%', 'FT%', 'AST', 'PTS']].copy()
New_Season_Stats_df.head()

,Player,id,G,FG%,FT%,AST,PTS
0,Curly Armstrong,0,63.0,0.28,0.71,176.0,458.0
1,Cliff Barker,1,49.0,0.37,0.71,109.0,279.0
2,Leo Barnhorst,2,67.0,0.35,0.70,140.0,438.0
3,Ed Bartels,3,15.0,0.26,0.56,20.0,63.0
4,Ed Bartels,4,13.0,0.26,0.55,20.0,59.0


In [212]:
# Create a filtered dataframe from specific columns
New_Stats = ["Player","id", "G", "FG%", "FT%", "AST", "PTS"]
New_Stats_transformed= New_Season_Stats_df[New_Stats].copy()

# Rename the column headers
New_Stats_transformed = New_Stats_transformed.rename(columns={"Player": "Player_Name",
                                                              "id": "id",
                                                              "G": "Games_Played",
                                                              "FG%": "Field_Goal_Percentage",
                                                              "FT%": "Free_Throw_Percentage",
                                                              "AST": "Total_Assists",
                                                              "PTS": "Total_Points"})

# Clean the data by dropping duplicates and setting the index
New_Stats_transformed.drop_duplicates(inplace=True)
New_Stats_transformed.set_index('Player_Name', inplace=True)

New_Stats_transformed.head()

,id,Games_Played,Field_Goal_Percentage,Free_Throw_Percentage,Total_Assists,Total_Points
Player_Name,,,,,,
Curly Armstrong,0,63.0,0.28,0.71,176.0,458.0
Cliff Barker,1,49.0,0.37,0.71,109.0,279.0
Leo Barnhorst,2,67.0,0.35,0.70,140.0,438.0
Ed Bartels,3,15.0,0.26,0.56,20.0,63.0
Ed Bartels,4,13.0,0.26,0.55,20.0,59.0


In [213]:
connection_string = ("root:Men15ard@localhost:3306/nba_db")
engine = create_engine(f'mysql://{connection_string}')

In [214]:
# Confirm tables
engine.table_names()

['Players', 'Season_Stats']

In [215]:
New_Stats_transformed.to_sql(name='Season_Stats', con=engine, if_exists='append', index=True)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '0' for key 'PRIMARY'")
[SQL: INSERT INTO `Season_Stats` (`Player_Name`, id, `Games_Played`, `Field_Goal_Percentage`, `Free_Throw_Percentage`, `Total_Assists`, `Total_Points`) VALUES (%(Player_Name)s, %(id)s, %(Games_Played)s, %(Field_Goal_Percentage)s, %(Free_Throw_Percentage)s, %(Total_Assists)s, %(Total_Points)s)]
[parameters: ({'Player_Name': 'Curly Armstrong', 'id': 0, 'Games_Played': 63.0, 'Field_Goal_Percentage': 0.28, 'Free_Throw_Percentage': 0.71, 'Total_Assists': 176.0, 'Total_Points': 458.0}, {'Player_Name': 'Cliff Barker', 'id': 1, 'Games_Played': 49.0, 'Field_Goal_Percentage': 0.37, 'Free_Throw_Percentage': 0.71, 'Total_Assists': 109.0, 'Total_Points': 279.0}, {'Player_Name': 'Leo Barnhorst', 'id': 2, 'Games_Played': 67.0, 'Field_Goal_Percentage': 0.35, 'Free_Throw_Percentage': 0.7, 'Total_Assists': 140.0, 'Total_Points': 438.0}, {'Player_Name': 'Ed Bartels', 'id': 3, 'Games_Played': 15.0, 'Field_Goal_Per

In [216]:
new_players_transformed.to_sql(name='Players', con=engine, if_exists='append', index=True)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '0' for key 'PRIMARY'")
[SQL: INSERT INTO `Players` (`Player_Name`, id, college) VALUES (%(Player_Name)s, %(id)s, %(college)s)]
[parameters: ({'Player_Name': 'Curly Armstrong', 'id': 0, 'college': 'Indiana University'}, {'Player_Name': 'Cliff Barker', 'id': 1, 'college': 'University of Kentucky'}, {'Player_Name': 'Leo Barnhorst', 'id': 2, 'college': 'University of Notre Dame'}, {'Player_Name': 'Ed Bartels', 'id': 3, 'college': 'North Carolina State University'}, {'Player_Name': 'Ralph Beard', 'id': 4, 'college': 'University of Kentucky'}, {'Player_Name': 'Gene Berce', 'id': 5, 'college': 'Marquette University'}, {'Player_Name': 'Charlie Black', 'id': 6, 'college': 'University of Kansas'}, {'Player_Name': 'Nelson Bobb', 'id': 7, 'college': 'Temple University'}  ... displaying 10 of 3922 total bound parameter sets ...  {'Player_Name': 'Paul Zipser', 'id': 3920, 'college': None}, {'Player_Name': 'Ivica Zubac', 'id': 3921